In [1]:
from __future__ import print_function
import argparse
import easydict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [5]:
args = easydict.EasyDict({
    "batch_size": 64,
    "test_batch_size": 1000,
    'epochs': 10,
    'lr': 0.01,
    'momentum': 0.5,
    'no_cuda': False,
    'seed': 1,
    'log_interval': 10,
})

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)
    torch.save(model, str(epoch))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.331558
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.329503
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.280593
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.340513
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.225874
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.205444
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.229443
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.198153
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.109512
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.997054
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.950289
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.917244
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.706681
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.671293
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.701643
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.406949
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.290307
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.311101
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.308394
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.192248
Train Epoch: 1 [12800/60000 (

/Users/DannyShau/anaconda/envs/py3.6/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 2 [640/60000 (1%)]	Loss: 0.672982
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.630493
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.419777
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.322363
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.500038
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.406658
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.319349
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.525267
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.459857
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.608163
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.573098
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.437738
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.509805
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.397699
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.532215
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.424282
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.477130
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.502344
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.430883
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.398997
Train Epoch: 2 [13440/60

In [6]:
the_model = torch.load(str(epoch))
test(args, model, device, test_loader)


Test set: Average loss: 0.0499, Accuracy: 9834/10000 (98%)

